# Omidyar Extractives Project 1
## OCR Contracts Backlog (Notebook 3 of 8)
### From directory of PDFs, performs OCR and outputs txt files to same directory

In [1]:
import io, os
import json
import codecs, unicodedata, re, urlparse, urllib2
import time
from PIL import Image as PI
import pyocr
import pyocr.builders
from wand.image import Image

In [2]:
# Where you want to save the PDFspip 
destination_folder = 'contract_data/Contracts_Backlog/'

In [3]:
pdfs = [unicodedata.normalize('NFKC',f.decode('utf8')) for f in os.listdir(destination_folder) if f.lower().endswith('.pdf')]

### Perform OCR on PDFs

In [4]:
def ocr_pdf_to_text(filename):
    tool = pyocr.get_available_tools()[0]
    lang = 'spa'
    req_image = []
    final_text = []
    image_pdf = Image(filename=filename, resolution=300)
    image_jpeg = image_pdf.convert('jpeg')
    for img in image_jpeg.sequence:
        img_page = Image(image=img)
        req_image.append(img_page.make_blob('jpeg'))

    for img in req_image: 
        txt = tool.image_to_string(
            PI.open(io.BytesIO(img)),
            lang=lang,
            builder=pyocr.builders.TextBuilder()
        )
        final_text.append(txt)
    return final_text

In [ ]:
errors = []
for filename in pdfs:
    txt_filename = destination_folder + filename[:-3] +'txt'
    try:
        ocr_txt = ocr_pdf_to_text(destination_folder + filename)
        with open(txt_filename,'w') as f:
            for i in range(len(ocr_txt)):
                f.write(json.dumps({i:ocr_txt[i].encode('utf8')}))
                f.write('\n')
        f.close()
    except:
        errors.append(filename) 

In [ ]:
len(ocr_txt)